In [2]:
pip install bs4


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [5]:
import requests

url = "https://www.niser.ac.in/people/fac/sbs"  # replace with real page
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
html = response.text

print(html[:500])

<!DOCTYPE html>
<html lang="en">
<head>
  <title>NISER</title>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta name="MobileOptimized" content="width">
  <meta name="HandheldFriendly" content="true">
  <meta name="robots" content="nocache, index, nofollow">
  <meta name="description" content="National Institute of Science Education and Research Bhubaneswar">
  <meta name="keywords" content="research, autonomous, dae, central gove


In [6]:
soup = BeautifulSoup(html, "lxml")

In [7]:
rows = soup.select("div.card-body")
print(f"Found {len(rows)} rows")


Found 28 rows


In [8]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

base_url = "https://www.niser.ac.in"
faculty_url = base_url + "/people/fac/sbs"

faculty_list = []

# Step 1: Fetch main faculty page
resp = requests.get(faculty_url, timeout=10)
soup = BeautifulSoup(resp.text, "lxml")

# Each faculty card
for card in soup.select("div.card-body.text-start"):
    # Name + profile link
    name_tag = card.select_one("h5 a")
    name = name_tag.get_text(strip=True) if name_tag else ""
    profile = base_url + name_tag["href"] if name_tag and name_tag.has_attr("href") else ""

    # Title / designation
    title_tag = card.select_one("p.color-gray")
    title = title_tag.get_text(" ", strip=True) if title_tag else ""

    # Email (reconstructed from pieces)
    email = ""
    mail_font = card.select_one("font[color='tomato']")
    mail_span = card.select_one("span.f12")
    if mail_font and mail_span:
        email = f"{mail_font.get_text(strip=True)}@{mail_span.get_text(strip=True)}"

    faculty_list.append({
        "name": name,
        "profile": profile,
        "title": title,
        "email": email
    })

print(f"Found {len(faculty_list)} faculty entries")

for faculty in faculty_list:
    profile_url = faculty.get("profile", "")
    if profile_url:
        # Handle relative URLs
        if profile_url.startswith("/"):
            profile_url = base_url + profile_url

        if not profile_url.startswith("http"):
            profile_url = "https://" + profile_url

        try:
            resp = requests.get(profile_url, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, "lxml")

            phd_text = ""
            postdoc_text = ""

            # Find the Education accordion
            education_div = soup.select_one("div.accordion-body")
            if education_div:
                for p in education_div.find_all("p"):
                    text = p.get_text(strip=True)
                    if "PhD" in text or "Ph.D." in text or "PhD thesis work" in text:
                        phd_text = text
                    elif "Postdoc" in text or "Postdoctoral" in text or "Post doctoral" in text:
                        postdoc_text = text

            faculty["phd"] = phd_text
            faculty["postdoc"] = postdoc_text

        except Exception as e:
            print(f"Error fetching {profile_url}: {e}")
            faculty["phd"] = ""
            faculty["postdoc"] = ""
    else:
        faculty["phd"] = ""
        faculty["postdoc"] = ""



Found 28 faculty entries


In [9]:
df = pd.DataFrame(faculty_list)
df.to_csv("nisersbs_faculty.csv", index=False, encoding="utf-8")

print(f"Saved {len(df)} faculty entries to nisersbs_faculty.csv")

Saved 28 faculty entries to nisersbs_faculty.csv
